In [1]:
import numpy as np
import xarray as xr
import dask
import zarr

### Load dataset

In [2]:
ds_int = xr.open_zarr('/rigel/ocp/users/jt2796/eccov4r3_datasets/anom/budgH_aveZ/intT/0300m')

### Find slope and intercept of linear trend

In [3]:
def linear_fit(y,t):
    y = y.reshape(-1)
    return np.polyfit(t,y,1).reshape(1,1,1,-1)

In [4]:
t = ds_int.time.astype(int)/1e9/3600/24/365.25

In [5]:
ds_slope = xr.Dataset(data_vars={})
ds_itcpt = xr.Dataset(data_vars={})

for varname in ds_int.data_vars:
    y = ds_int[varname].transpose('face','j','i','time').chunk(chunks=(1,1,1,-1))
    fit = dask.array.map_blocks(linear_fit, y.data, t, dtype=np.float64, chunks=(1,1,1,2))
    ds_slope[varname] = xr.DataArray(fit[:,:,:,0],coords=[ds_int.face,ds_int.j,ds_int.i],dims=['face','j','i'])
    ds_itcpt[varname] = xr.DataArray(fit[:,:,:,1],coords=[ds_int.face,ds_int.j,ds_int.i],dims=['face','j','i'])

### Save slope and intercept to netcdf

In [6]:
ds_itcpt

<xarray.Dataset>
Dimensions:           (face: 13, i: 90, j: 90)
Coordinates:
  * face              (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * j                 (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * i                 (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    frc               (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    hadv_Uanom_Tanom  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    hadv_Uanom_Tclim  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    hadv_Uclim_Tanom  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    hdif              (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    res               (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    tnd               (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    vadv_Uanom_Tan

#### Rechunk and compression

In [7]:
opts = {}
for varname in ds_itcpt.data_vars:
    ds_itcpt[varname] = ds_itcpt[varname].chunk(chunks={'face':13,'j':90,'i':90})
    opts[varname] = {'zlib': True}

#### Save to file

In [8]:
from dask.diagnostics import ProgressBar

In [ ]:
with ProgressBar():
    ds_itcpt.to_netcdf('/rigel/ocp/users/jt2796/eccov4r3_datasets/anom/budgH_aveZ/intT/linear_fit/itcpt/0300.nc',
                       format='NETCDF4',encoding=opts)

/rigel/home/jt2796/dask/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


[###                                     ] | 8% Completed |  1hr 23min 12.3s

In [6]:
# Linear trend dataset
ds_trn = ds_itcpt + ds_slope*t

In [7]:
# Detrended dataset
ds_dtr = ds_int - ds_trn